In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from docx import Document
from docx.shared import Inches
from docx.shared import Pt
import io

df = pd.read_csv("pol.csv")

SN = df.iloc[:, [0]]  #SN
Location = df.iloc[:, [1]]  #location
Parent_location = df.iloc[:, [2]]  # parent location
Fac_area = df.iloc[:, [3]]  
Dev_typ = df.iloc[:, [4]]  
line_neutral = df.iloc[:, [5]]  
Type_Supply = df.iloc[:, [6]]  
Polarity_Reference = df.iloc[:, [7]]  





def result(line_neutral):
    if line_neutral == 230:
        return "OK"
    else :
        return "REVERSE"
    

def rang(length):
    res = []
    for row in range(0, length):
        line_neutral = df.iloc[row, 5]
        res.append(result(line_neutral))
    return res


def create_table(df, doc):
    table_data = df.iloc[:, 0:]
    num_rows, num_cols = table_data.shape
    table = doc.add_table(rows=num_rows + 1, cols=num_cols + 1)
    table.style = "Table Grid"
    table.autofit = False
    column_widths = {
        0: 0.2,
        1: 0.51,
        2: 0.55,
        3: 0.54,
        4: 0.38,
        5: 0.56,
        6: 0.5,
        7: 0.48,
        8: 0.71,
       
    }
    for j, col in enumerate(table_data.columns):
        table.cell(0, j).text = col
        table.cell(0, j).width = Inches(column_widths[j])
    for i, row in enumerate(table_data.itertuples(), start=1):
        for j, value in enumerate(row[1:], start=0):
            table.cell(i, j).text = str(value)
    Results = rang(num_rows)
    table.cell(0, num_cols).text = "Result"
    table.cell(0, num_cols).width = Inches(0.8)
    for i in range(num_rows):
        res_index = i
        table.cell(i + 1, num_cols).text = Results[res_index]
    font_size = 7

    for row in table.rows:
        for cell in row.cells:
            for paragraph in cell.paragraphs:
                for run in paragraph.runs:
                    run.font.size = Pt(font_size)
    return doc


# def graph(df):
#     x = df["Line to Neutral Voltage (V)"]
#     y = df["Polarity Reference"]
#     plt.bar(x, y)
#     plt.xlabel("Line to Neutral Voltage (V)")
#     plt.ylabel("Polarity Reference")
#     plt.title("Calculated Voltage Drop (V) by Voltage Drop %")
#     graph = io.BytesIO()
#     plt.savefig(graph)
#     plt.close()
#     return graph


def main():
    doc = Document()
    doc.add_heading("POLARITY TEST", 0)
    doc = create_table(df, doc)
    # histo_graph = graph(df)
    # doc.add_picture(histo_graph, width=Inches(6))
    doc.save("polarity.docx")


main()

FileNotFoundError: [Errno 2] No such file or directory: 'pol.csv'

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import docx
from docx import Document
import csv
import io
from docx.shared import Inches
from docx.enum.section import WD_SECTION
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.shared import Pt, RGBColor
import numpy as np
import datetime
from docx.oxml.ns import nsdecls
from docx.oxml import parse_xml
from docx.shared import RGBColor
from docx.oxml.ns import nsdecls
from docx.oxml import parse_xml


H= "func_ops.csv"
of = pd.read_csv(H)

def func_ops_result(func_chk,inter_chk):
    if func_chk == "OK" and inter_chk == "OK":
        return "pass"
    elif func_chk == "OK" and inter_chk == "Not OK":
        return "fail"
    elif func_chk == "Not OK " and inter_chk == "OK":
        return "fail"
    elif func_chk == "Not OK" and inter_chk == "OK":
        return "fail"
    else:
        return "Invalid"
    

def func_ops_table(of, doc):

    table_data = of.iloc[:, 0:]
    num_rows, num_cols = table_data.shape[0], table_data.shape[1]
    table = doc.add_table(rows=num_rows + 1, cols=num_cols)
    num_rows, num_cols = table_data.shape
    table.style = "Table Grid"
    table.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT
    table.autofit = False
    column_widths = {
        0: 0.2,
        1: 0.51,
        2: 0.55,
        3: 0.54,
        4: 0.38,
        5: 0.56,
        6: 0.5,
        7: 0.48,
        8: 0.71,
        9: 0.43,
        10: 0.56,
        11: 0.56, 
        12:0.5   # Add this line
    }
    for j, col in enumerate(table_data.columns):
        table.cell(0, j).text = col
        table.cell(0, j).width = Inches(column_widths.get(j, 1))
        table.cell(0, j).paragraphs[0].alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

    for j, col in enumerate(table_data.columns):
        table.cell(0, j).text = col
        table.cell(0, j).width = Inches(column_widths[j])
        table.cell(0, j).paragraphs[0].alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
        first_row_cells = table.rows[0].cells
        for cell in first_row_cells:
            cell.paragraphs[0].alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
            cell_elem = cell._element
            tc_pr = cell_elem.get_or_add_tcPr()
            shading_elem = parse_xml(f'<w:shd {nsdecls("w")} w:fill="d9ead3"/>')
            tc_pr.append(shading_elem)

    for i, row in enumerate(table_data.itertuples(), start=1):
        for j, value in enumerate(row[1:], start=0):
            cell = table.cell(i, j)
            cell.text = str(value)
            cell.paragraphs[0].alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
            if j == num_cols - 1:  # Apply background color only to the Result column
                result_cell = cell
                if value.startswith("PASS"):
                    shading_elm = parse_xml(
                        r'<w:shd {} w:fill="#5ac85a"/>'.format(nsdecls("w"))
                    )  # Green color
                    result_cell._tc.get_or_add_tcPr().append(shading_elm)
                elif value.startswith("FAIL"):
                    shading_elm = parse_xml(
                        r'<w:shd {} w:fill="#dc0000"/>'.format(nsdecls("w"))
                    )  # Red color
                    result_cell._tc.get_or_add_tcPr().append(shading_elm)
    for section in doc.sections:
        section.left_margin = Inches(0.2)

    font_size = 7
    for row in table.rows:
        for cell in row.cells:
            for paragraph in cell.paragraphs:
                for run in paragraph.runs:
                    run.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
                    run.font.size = Pt(font_size)
                    run.font.name = "Calibri"

    return doc

def func_ops_combined_graph(of):
    plt.figure(figsize=(16, 8))

    # Bar graph
    plt.subplot(121)
    x = of["Interlock check"]
    y = of["SN"]
    colors = ["#d9534f", "#5bc0de", "#5cb85c", "#428bca"]
    plt.bar(x, y, color=colors)
    plt.xlabel("Interlock check")
    plt.ylabel("SN")
    plt.title("SN VS Interlock check")

    # Pie chart
    plt.subplot(122)
    result_counts = of["Result"].value_counts()
    labels = result_counts.index
    values = result_counts.values
    colors = ["#5ac85a", "#dc0000"]
    plt.pie(values, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
    plt.title("Earth Pit Electrode Test Results (Pie Chart)")
    plt.axis('equal')
    graph_combined = io.BytesIO()
    plt.savefig(graph_combined)
    plt.close()


    return graph_combined

def main():
    doc = Document()
    doc.add_heading('RESISTANCE CONDUCTOR TEST', 0)
    doc = func_ops_table(of, doc)
    graph_combined = func_ops_combined_graph(of)
    doc.add_picture(graph_combined, width=Inches(8), height=Inches(4))
    doc.save("funcandops.docx")


main()

FileNotFoundError: [Errno 2] No such file or directory: 'func_ops.csv'